## CONTRADICTORY, MY DEAR WATSON

### In order for the BERT model to work, this notebook must be run in a GPU setting. I used Google Colabs to run the NB to train the BERT model and obtained a 67% accuracy.  

In [183]:
#import libraries
import numpy as np 
import pandas as pd

SyntaxError: invalid syntax (3001639507.py, line 2)

In [144]:
#csv file
df=pd.read_csv('~/Downloads/train.csv')

In [145]:
english= df

In [146]:
train=english[['premise', 'hypothesis', 'label']]

In [10]:
train.shape

(6870, 3)

In [168]:
train

,premise,hypothesis,label,combined
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0,[CLS]and these comments were considered in for...
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2,[CLS]These are issues that we wrestle with in ...
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0,[CLS]Des petites choses comme celles-là font u...
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0,[CLS]you know they can't really defend themsel...
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1,[CLS]ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะไ...
...,...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2,[CLS]The results of even the most well designe...
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0,[CLS]But there are two kinds of the pleasure ...
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2,[CLS]The important thing is to realize that it...
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2,[CLS]At the west end is a detailed model of th...


In [151]:
train

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1
...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


### USE CLS

In [153]:
train.premise[1]

'These are issues that we wrestle with in practice groups of law firms, she said. '

In [178]:
#The tokenizer and TFBert Model is initialized with the language model 
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

#function to add [SEP] in between premise and hypothesis
def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [158]:
encode_sentence

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.37029105, -0.0800719 ]], dtype=float32)>

In [179]:
#encode_sentence and than add [CLS] to beginning of each sentence. Since BERT takes in input_word_ids, input_mask 
#and input_type_ids, I convert the sentences into these and turn into tensor
def bert_encode(hypotheses, premises, tokenizer):
    
    num_examples = len(hypotheses)
  
    sentence1 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(hypotheses)])
    sentence2 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(premises)])

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor()

    inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs

In [161]:
#combine all to create my X as input
train_input = bert_encode(train.premise.values, train.hypothesis.values, tokenizer)

In [180]:
max_len = 259

def build_model():
    bert_encoder = TFBertModel.from_pretrained('bert-base-multilingual-cased')
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [181]:

model = build_model()
model.summary()


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 259)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 259)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 259)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_5 (TFBertModel)   TFBaseModelOutputWit 177853440   input_word_ids[0][0]             
                                                                 input_mask[0][0]          

In [182]:
# the model must be fitted using Google Colabs. I do not have GPU on my computer so it threw an error in my case!
model.fit(train_input, train.label.values, epochs = 2, verbose = 1, batch_size = 64, validation_split = 0.2, workers=-1)

Epoch 1/2


InvalidArgumentError: Cannot assign a device for operation model_10/tf_bert_model_5/bert/embeddings/Gather: Could not satisfy explicit device specification '' because the node {{colocation_node model_10/tf_bert_model_5/bert/embeddings/Gather}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSubVariableOp: GPU CPU 
RealDiv: GPU CPU 
Sqrt: GPU CPU 
UnsortedSegmentSum: GPU CPU 
AssignVariableOp: GPU CPU 
ReadVariableO